# Load and vehicle capacities

TODO

### Capacitated VRP

We will first model and solve the small capacitated VRP instance with 16 clients defined [in the OR-Tools documentation](https://developers.google.com/optimization/routing/cvrp).
This instance has an optimal solution of cost 6208.
The data are as follows:

In [ ]:
# fmt: off
COORDS = [
    (456, 320),  # location 0 - the depot
    (228, 0),    # location 1
    (912, 0),    # location 2
    (0, 80),     # location 3
    (114, 80),   # location 4
    (570, 160),  # location 5
    (798, 160),  # location 6
    (342, 240),  # location 7
    (684, 240),  # location 8
    (570, 400),  # location 9
    (912, 400),  # location 10
    (114, 480),  # location 11
    (228, 480),  # location 12
    (342, 560),  # location 13
    (684, 560),  # location 14
    (0, 640),    # location 15
    (798, 640),  # location 16
]
DEMANDS = [0, 1, 1, 2, 4, 2, 4, 8, 8, 1, 2, 1, 2, 4, 4, 8, 8]
# fmt: on

We can use the `pyvrp.Model` interface to conveniently specify our vehicle routing problem using this data.
A full description of the `Model` interface is given in our [API documentation](https://pyvrp.org/api/pyvrp.html#pyvrp.Model.Model).

In [ ]:
from pyvrp import Model

m = Model()
m.add_vehicle_type(4, capacity=15)
depot = m.add_depot(x=COORDS[0][0], y=COORDS[0][1])
clients = [
    m.add_client(x=COORDS[idx][0], y=COORDS[idx][1], delivery=DEMANDS[idx])
    for idx in range(1, len(COORDS))
]

for frm in m.locations:
    for to in m.locations:
        distance = abs(frm.x - to.x) + abs(frm.y - to.y)  # Manhattan
        m.add_edge(frm, to, distance=distance)

Let's inspect the resulting data instance.

In [ ]:
import matplotlib.pyplot as plt

from pyvrp.plotting import plot_coordinates

_, ax = plt.subplots(figsize=(8, 8))
plot_coordinates(m.data(), ax=ax)

The instance looks good, so we are ready to solve it.
Let's do so with a second of runtime, and display the search progress using the `display` argument on `Model.solve`.

In [ ]:
from pyvrp.stop import MaxRuntime

res = m.solve(stop=MaxRuntime(1), display=True)  # one second

By passing the `display` argument, PyVRP displays statistics about the solver progress and the instance being solved.
In particular, it outputs the sizes of the feasible and infeasible solution pools, their average objective values, and the objective of the best solutions in either pool.
A heuristic improvement is indicated by a `H` at the start of a line.

Let's print the solution we have found to see the routes.

In [ ]:
print(res)

Good! Our solution attains the same objective value as the optimal solution OR-Tools finds.
Let's inspect our solution more closely.

In [ ]:
from pyvrp.plotting import plot_solution

_, ax = plt.subplots(figsize=(8, 8))
plot_solution(res.best, m.data(), ax=ax)

We have just solved our first vehicle routing problem using PyVRP!

.. warning::
   PyVRP automatically converts all numeric input values to integers, except for client and depot (x, y) coordinates.
   If your data has decimal values, you must scale and convert them to integers first to avoid unexpected behaviour.


### VRP with simultaneous pickup and delivery

We will now consider the VRP with simultaneous pickup and delivery.
In this problem variant, clients request items from the depot, and also produce return shipments that needs to be delivered back to the depot after visiting the client.
Thus, there are both deliveries from the depot to the clients, and pickups from the clients to the depot.

Let's remain in the multi-depot, prize-collecting world we entered through the last example.
We first define a `LOADS` list that tracks the delivery and pickup amount for each location:

In [ ]:
# fmt: off
LOADS = [
    (0, 0),   # location 0 - a depot
    (0, 0),   # location 1 - a depot
    (1, 4),   # location 2 - simultaneous pickup and delivery
    (2, 0),   # location 3 - pure delivery
    (0, 5),   # location 4 - pure pickup
    (6, 3),   # location 5 - simultaneous pickup and delivery
    (4, 7),   # location 6 - simultaneous pickup and delivery
    (11, 0),  # location 7 - pure delivery
    (3, 0),   # location 8 - pure delivery
    (0, 5),   # location 9 - pure pickup
    (6, 4),   # location 10 - simultaneous pickup and delivery
    (1, 4),   # location 11 - simultaneous pickup and delivery
    (0, 3),   # location 12 - pure pickup
    (6, 0),   # location 13 - pure delivery
    (3, 2),   # location 14 - simultaneous pickup and delivery
    (4, 3),   # location 15 - simultaneous pickup and delivery
    (0, 6),   # location 16 - pure pickup
]
# fmt: on

In [ ]:
m = Model()

for idx in range(2):
    depot = m.add_depot(x=COORDS[idx][0], y=COORDS[idx][1])
    m.add_vehicle_type(
        2,
        start_depot=depot,
        end_depot=depot,
        capacity=15,
        tw_early=TIME_WINDOWS[idx][0],
        tw_late=TIME_WINDOWS[idx][1],
    )


for idx in range(2, len(COORDS)):
    m.add_client(
        x=COORDS[idx][0],
        y=COORDS[idx][1],
        tw_early=TIME_WINDOWS[idx][0],
        tw_late=TIME_WINDOWS[idx][1],
        delivery=LOADS[idx][0],
        pickup=LOADS[idx][1],
        prize=PRIZES[idx],
        required=False,
    )

for frm_idx, frm in enumerate(m.locations):
    for to_idx, to in enumerate(m.locations):
        distance = abs(frm.x - to.x) + abs(frm.y - to.y)  # Manhattan
        duration = DURATION_MATRIX[frm_idx][to_idx]
        m.add_edge(frm, to, distance=distance, duration=duration)

In [ ]:
res = m.solve(stop=MaxRuntime(1), display=False)  # one second
print(res)

In [ ]:
_, ax = plt.subplots(figsize=(8, 8))
plot_solution(res.best, m.data(), plot_clients=True, ax=ax)